In [1]:
!pip install -e ../.

Looking in indexes: https://pypi.org/simple, https://pypi.org/simple
Obtaining file:///Users/nelenj/Documents/projects/llmex
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for helmet (pyproject.toml) ... done
  Created wheel for helmet: filename=helmet-0.1.2-py3-none-any.whl size=1747 sha256=c0b60f635fd34f044790856ba90f244682db1c11f3be0acd0dc31c7081136139
  Stored in directory: /private/var/folders/f1/st6fs1q547zbfmzy6jr5b0km0000gp/T/pip-ephem-wheel-cache-9t000iqx/wheels/8f/ce/ec/788efcbd8ae032d4d5999926126a0fac8b4f2ad3adcf38c2ed
Successfully built helmet
  Attempting uninstall: helmet
    Found existing installation: helmet 0.1.2
    Uninstalling helmet-0.1.2:
      Successfully uninstalled helmet-0.1.2


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# This to make sure that the changes in the module are reloaded
%reload_ext autoreload

In [9]:
# Platform configs
url = "https://api.jeroennelen.nl"
project_id = "66277759e1eacb39208e10ed"

project_setup = {
    "platform_url": url,
    "project_id": project_id
}

In [10]:
# checkpoint = "mlabonne/Gemmalpaca-2B"
# "meta-llama/Meta-Llama-3-8B"
device = "cpu"
checkpoint = "bigscience/bloom-1b7"

model_setup = {
    "checkpoint": checkpoint,
    "model_type": "dec",
    "embeddings": "transformer.word_embeddings"
}

run_config = {
    "device": device,
    "max_new_tokens": 100,
    # "temperature": 1, # Not
    # e that if you add this, then you need to add "do_sample": True
    # "do_sample": True
}

In [11]:
import helmet

In [12]:
model = helmet.from_pretrained(project_setup=project_setup, model_setup=model_setup, run_config=run_config)

updates will be sent to https://api.jeroennelen.nl
setting up model with config {'checkpoint': 'bigscience/bloom-1b7', 'model_type': 'dec', 'embeddings': 'transformer.word_embeddings'}
model loaded


In [ ]:
# !pip install huggingface_hub ipywidgets -q

In [ ]:
# This might be needed to load the model
# from huggingface_hub import login
# login()

In [5]:
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", name="en-nl", split='test[:10]')

In [6]:
dataset[1]

{'translation': {'en': '- How long have I slept?',
  'nl': 'Hoe lang heb ik geslapen?'}}

In [ ]:
# prompt = dataset["validation"][1]
# # print(prompt)
# # prompt = prompt["question"]

In [7]:
source_lang = "en"
target_lang = "nl"
prefix = "Translate English to Dutch: "

def prompt_builder(sample):
    return prefix + sample['translation'][source_lang]


prompt = prompt_builder(dataset[1])
prompt

'Translate English to Dutch: - How long have I slept?'

In [13]:
predict_config = {
    "max_new_tokens": 10,
    "generate_explanations": False,
}

# prompt = "Translate English to Italian: I am a student."
result = model.predict(prompt, **predict_config)
result

updated app, result:  {'acknowledged': True, 'insertedId': '6627781fe1eacb39208e10ee'}


' - How many hours have I slept? - How'

In [ ]:
model.saliency_explainer("662776cdc9a6d6bd85c6d398")